All files contain a (possibly infeasible) system Ax <= b with the
following scheme (all fields are separated by a tab):

1) First line: number of variables n, number of constraint m, number
of nonzeros nnz;

2) Lines 2 to n+1: each line contains lower and upper bound of all
variables (-/+ 1e30 if unbounded);

3) lines n+2 + n+m+1: each line describes an inequality with K terms 

$$l <= \sum_{k=1..K} a_{i(k)} x_{i(k)} <= u 
$$
with a list of numbers (separated by tab) such that:

  - the first number is K, the number of variables in the inequality

  - the second and third are l and u, respectively. Note: either
    l=-1e30 or u=1e30, i.e., for now Creme only deals with
    inequalities;

  - the remaining 2K elements are pairs (index,coefficient), i.e.,
    $i(k)$ and $a_{i(k)}$

A translator from .lp to this compact format is in the works and will
be included in an early revision of Creme.

In [1]:
import numpy as np
from io import StringIO
import gurobipy as gp
from gurobipy import GRB
data = []
path='./Data/'
name='monks-train-124'
for line in open(path+name):
    data.append(np.genfromtxt(StringIO(line.replace('\n','')),delimiter='\t',dtype=str))
data
#np.genfromtxt(delimiter='\t',dtype=str)

[array(['8', '124', '992', '0'], dtype='<U3'),
 array(['-1', '1'], dtype='<U2'),
 array(['-1', '1'], dtype='<U2'),
 array(['-1', '1'], dtype='<U2'),
 array(['-1', '1'], dtype='<U2'),
 array(['-1', '1'], dtype='<U2'),
 array(['-1', '1'], dtype='<U2'),
 array(['-1e30', '1e30'], dtype='<U5'),
 array(['0.001', '1e30'], dtype='<U5'),
 array(['8', '-1e30', '0', '0', '0.33333', '1', '0.66667', '2', '0.5', '3',
        '0.33333', '4', '0.5', '5', '0.5', '6', '-1', '7', '1'],
       dtype='<U7'),
 array(['8', '-1e30', '0', '0', '0.33333', '1', '0.66667', '2', '0.5', '3',
        '0.33333', '4', '0.75', '5', '0.5', '6', '-1', '7', '1'],
       dtype='<U7'),
 array(['8', '-1e30', '0', '0', '0.33333', '1', '0.66667', '2', '0.5', '3',
        '0.33333', '4', '1', '5', '1', '6', '-1', '7', '1'], dtype='<U7'),
 array(['8', '-1e30', '0', '0', '0.33333', '1', '0.66667', '2', '0.5', '3',
        '0.66667', '4', '0.75', '5', '0.5', '6', '-1', '7', '1'],
       dtype='<U7'),
 array(['8', '-1e30', '0', '0'

In [11]:
n = gp.Model(name)
number_of_continuous = int(data[0][0])
number_of_row = int(data[0][1])
begin_index_of_inequality = int(data[0][0]) + 1
end_index_of_inequality = begin_index_of_inequality + int(data[0][1])
M = -1000
z = n.addVars(number_of_row, vtype=GRB.BINARY, name='z')
w = n.addVars(number_of_continuous, vtype=GRB.CONTINUOUS, name="w&c&d")  #定义决策变量
for i in range(0, number_of_continuous):
    w[i].lb = float(data[i + 1][0])
    w[i].ub = float(data[i + 1][1])
for i in range(begin_index_of_inequality, end_index_of_inequality):
    n.addConstr(
                gp.quicksum(float(data[i][2 * k + 2]) * w[int(data[i][2 * k + 1])]for k in range(1,int(data[i][0]) + 1)) +
                M * z[i - begin_index_of_inequality] <= 0)
n.setObjective(gp.quicksum(z[i] for i in range(number_of_row)), GRB.MINIMIZE)
n.update()
#n.optimize()

In [12]:
def mycallback(model, where):
    if where == GRB.Callback.MIPNODE:
        # MIP node callback
        if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.INFEASIBLE:
            sol = model.cbGetNodeRel(model._vars)
            sol=sol[0:number_of_row]
            support_all = generate_support(sol)
            print(sol)
            for support in support_all:
                model.cbLazy(
                    gp.quicksum(model._vars[i] if sol[i] == 0 else 0
                                for i in support) +
                    gp.quicksum((1 - model._vars[i]) if sol[i] == 1 else 0
                                for i in support) >= 1)
n._vars = n.getVars()
n.Params.lazyConstraints = 1
n.optimize(mycallback)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 124 rows, 132 columns and 1116 nonzeros
Model fingerprint: 0xd7f215e1
Variable types: 8 continuous, 124 integer (124 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-03, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 62.0000000
Presolve time: 0.00s
Presolved: 124 rows, 132 columns, 1116 nonzeros
Variable types: 8 continuous, 124 integer (124 binary)

Root relaxation: objective 7.733327e-05, 151 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000773    0.00008  0.00%     -    0s

Exp

In [5]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from Import_Data import import_by_name

#names
# BCW683           10
# busvan-445       8
# chorales-107     27
# lymphography142  5
# monks-train-124  24 

name = 'monks-train-124'

A, number_of_row, number_of_continuous = import_by_name(name)
row_A, col_A = A.shape
Mz = -1000   # big-M coefficient

def generate_sub_support(m, x, b, row, sol_index):
    support_all = []
    x.ub = np.inf * np.ones(len(row))
    for i in range(-1, len(row)):
        if i != -1:
            x.ub = np.inf * np.ones(len(row))
            x[i].ub = 0
        m.optimize()
        status = m.status
        if status == GRB.OPTIMAL:
            # print('The optimal objective is %g' % m.objVal)
            support = list(map(lambda x: x[1] if x[0] != 0 else None, zip(x.x, [i for i in range(len(b))])))
            support = list(
                map(lambda x: (sol_index + [i for i in range(number_of_row, row_A)])[x] if x != None else None,
                    support))
            support = list(filter(lambda x: x != None and x < number_of_row, support))
            if (support not in support_all) and support !=[]:
                support_all.append(support)
        if status == GRB.INF_OR_UNBD or status == GRB.INFEASIBLE:
            # print('Optimization was stopped with status %d' % status)
            pass
    return support_all


def generate_support(sol):
    combination_filter = list(zip(*filter(lambda x: x[0]==1 or x[0]==0, zip(sol, [i for i in range(len(sol))]))))
    sol = list(combination_filter[0])
    sol_index = list(combination_filter[1])
    row = sol_index + [i for i in range(number_of_row, row_A)]
    k = 2
    b = np.array(list(map(lambda x: -Mz * x, sol))).reshape(-1, 1)
    b = np.vstack((b, np.ones((row_A - number_of_row - 1, 1))))
    b = np.vstack((b, -0.001))
    w = np.array([k for i in range(len(sol_index))] + [1 for i in range(number_of_row, row_A)])

    m = gp.Model()
    x = m.addMVar(shape=len(row), lb=0, vtype=GRB.CONTINUOUS, name="x")
    m.addConstr(A[row].T @ x == 0, name="c")
    m.addConstr(b.reshape(-1) @ x == -1, name='b')
    m.setObjective(w @ x, GRB.MINIMIZE)
    m.setParam('OutputFlag', 0)
    support_all = generate_sub_support(m, x, b, row, sol_index)
    return support_all


def mycallback(model, where):
    if where == GRB.Callback.MIPSOL:
        sol = model.cbGetSolution(model._vars)
        sol=[round(i) for i in sol]
        support_all = generate_support(sol)
        for support in support_all:
            model.cbLazy(
                gp.quicksum(model._vars[i] if sol[i] == 0 else 0
                            for i in support) +
                gp.quicksum((1 - model._vars[i]) if sol[i] == 1 else 0
                            for i in support) >= 1)
    elif where == GRB.Callback.MIPNODE:
        if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL and model.cbGet(GRB.Callback.RUNTIME)<60:
            sol = model.cbGetNodeRel(model._vars)
            support_all = generate_support(sol)
            for support in support_all:
                model.cbLazy(
                    gp.quicksum(model._vars[i] if sol[i] == 0 else 0
                                for i in support) +
                    gp.quicksum((1 - model._vars[i]) if sol[i] == 1 else 0
                                for i in support) >= 1)
    return 0


master = gp.Model(name)
z = master.addVars(number_of_row, vtype=GRB.BINARY, name='z')
master.setObjective(gp.quicksum(z[i] for i in range(number_of_row)), GRB.MINIMIZE)
master.update()
master._vars = master.getVars()
master.Params.lazyConstraints = 1
master.Params.ZeroHalfCuts = 2
master.optimize(mycallback)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter ZeroHalfCuts to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 124 columns and 0 nonzeros
Model fingerprint: 0x94a4f09e
Variable types: 0 continuous, 124 integer (124 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 124 columns, 0 nonzeros
Variable types: 0 continuous, 124 integer (124 binary)

Root relaxation: objective 1.500000e+00, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.50000    0    3          -    1.